## COMP30024 Artificial Intelligence Part A

Author: Rin Huang, Eric Wei

In this first part of the project, you will solve a simple search-based problem on the Cachex game board.
Before you read this specification, please make sure you have carefully read the entire ‘Rules for the Game
of Cachex’ document. Although you won’t be writing an agent to play the game just yet, you should be
aiming to get very familiar with the board layout and corresponding hex coordinate system.

The aims for Project Part A are for you and your project partner to
- refresh and extend your Python programming skills
- explore some of the algorithms you have encountered in lectures, and 
- become more familiar with the Cachex task environment.

This is also a chance for you to develop fundamental Python tools for working with the game: Some of the functions and classes you create now may be helpful later (when you are building your full game-playing program for Part B of the project).

To compute this path you are asked to use an A* search and design an **admissible heuristic** to optimise its performance. There are a number of assumptions you should make:
1. The start and goal cells will always be unoccupied (but any other cells may be occupied).
2. All given cell coordinates will be within the bounds of a board of size n. More precisely, for any given cell coordinate (r: row, q: column), 0 ≤ r < n and 0 ≤ q < n. You may also assume n ≥ 1.
3. The cost of a path is defined as the number of cells that form a continuous path from the start cell to the goal cell (including these cells).
4. If there is a tie, that is, multiple minimal paths of the same cost exist on the same board configuration, any such path is a valid solution.
5. There may not always be a valid path from the 

### Import required libraries & Define global variables

In [1]:
# import required libraries
import json
from math import pow, sqrt, fabs
from util import print_board
import time
from datetime import date

In [2]:
# define global variables
RED = 'Red'
BLUE = 'Blue'
BLOCK = 'Block'

In [3]:
# obtain board data via open the sample_input.json
# this should be read via system args
with open('sample_input.json') as json_file:
    # read cachex board game data as a const variable
    BOARD_DATA = json.load(json_file)

### Define CachexBoard Type

In [31]:
class CachexBoard:
    """
    Cachex Game object functions, a CachexBoard object should initialised with a 
    dictionary which contains board data:
    
    For example:
    BOARD_DATA = {
        'n': 5, # board dimensions
        'board': [['b', 1, 0], ['b', 1, 1], ['b', 3, 2], ['b', 1, 3]], # barriers on the game board
        'start': [4, 2], # node start point
        'goal': [0, 0] # node end point
    }
    
    board = CachexBoard(data=BOARD_DATA)
    
    The final object will contain the following attributes:
    - board.n: number of dimensions
    - board.start: board start point
    - board.goal: board target end point
    - board.data: restore board data
    - board.NodeDict: dictionary object contain each node information
    - board.board: game board layout
    - board.barriers: barriers on the board
    - board.display: print the current board
    """
    def __init__(self, data):
        self.n = data['n']
        
        # check current start and goal is valid
        if data['start'][0]+1 > self.n or data['start'][1]+1 > self.n:
            raise InvalidStartError
        self.start = (data['start'][0], data['start'][1])
        
        if data['goal'][0]+1 > self.n or data['goal'][1]+1 > self.n:
            raise InvalidGoalError
            
        self.goal = (data['goal'][0], data['goal'][1])
        self.data = data
        
        self.construct_board(self.n)
        self.construct_node_dict()
        self.obtain_barrier_coord()
    
    def construct_board(self, n: int, inplace=True):
        """
        The function will return all valid hexagon cell coordinates in a single
        set
        input: n: int # number of the board size
        return: board: set # a set of all possilble moves
        """
        self.board = set()

        # construct cachex board
        for r in range(BOARD_DATA['n']):
            for q in range(BOARD_DATA['n']):
                self.board.add((r, q))
        if not inplace:
            return board
    
    def construct_node_dict(self):
        """
        Construct a dictionary contain
        {
            (r[0], q[0]) : HexNode(), 
            (r[1], q[1]) : HexNode(),
            ...,
            (r[n-1], q[n-1]) : HexNode()
        }
        """
        
        self.NodeDict = dict()
        
        for node in self.board:
            # check node is barrier or not
            self.NodeDict[node] = HexNode(node)
            self.NodeDict[node].find_next_moves(self.board)
    
    def obtain_barrier_coord(self):
        """
        Read board data from the json dictionary.
        if a node has a letter 'b' at the position 0, then it is a barrier
        """
        self.barriers = set()
        
        for node in self.data['board']:
            if node[0] is 'b':
                self.NodeDict[(node[1], node[2])].state = BLOCK
                self.barriers.add((node[1], node[2]))
                
    def display(self, path=None):
        """
        This function will print the current board with the given board information
        """
        # define path dictionary and board dictionary
        path_dict = dict()
        if path is not None:
            for i, p in enumerate(path):
                path_dict[p] = i + 1 
            
        board_dict = {self.start: 'Δ', self.goal: '$'}
        for barrier in self.barriers:
            board_dict[barrier] = '#'
        
        # merge path_dict and board_dict into a single dict
        # note: path_dict only contain the path info include start and goal
        # therefore update path_dict by board_dict where board_dict
        # will rewrite start and goal with their unique symbol
        board_dict = {**path_dict, **board_dict}
       
        # define the message
        message = self.message(path)
                          
        # print game board
        print_board(n=self.n, board_dict=board_dict, message=message)
        
    def message(self, path, sep='\n'):
        """
        Message generator for display function
        """
        message = ["2022 COMP30024 Artificial Intelligence Cachex Game",
                   f"                       Group 4399 S Huang, W, Zhao",
                   "--------------------------------------------------",
                   "Symbol Representation:",
                   "- Δ: AStar Search Start Point",
                   "- $: AStar Search End/Goal Point",
                   "- #: Barriers, node cannot place at here",
                   f"- [1-{len(path)}]: AStar Path Result",
                   "--------------------------------------------------",
                   f"Board Information: Start: {self.start} >>> End: {self.goal}"]
        if path is not None:
            message.append(f"- AStar Path Length: {len(path)}")
            message.append("--------------------------------------------------")
            message.append("AStar Path:")
            
            path_str = ""
            for i, p in enumerate(path):
                if i == 0 or i % 4 != 0:
                    if p == self.start:
                        path_str += 'Start -> '
                    elif p == path[-1]:
                        path_str += "Goal"
                    else:
                        path_str += str(p) + ' -> '
                else:
                    path_str += str(p) + ' ->\n'
            message.append(path_str)
                        
        message.append("--------------------------------------------------")
        return sep.join(message)

### Define Custom Error

In [5]:
# define the error type
class Error(Exception):
    """
    Cachex AStar Path Solver Error
    """
    pass

class InvalidHeuristicError(Error):
    """
    Heuristic function must be one of the following distance formula:
    ['euclidean', 'manhatten', 'hamming']
    """
    def __init__(self):
        self.message = "Heuristic function must be one of the following distance formula: ['euclidean', 'manhatten', 'hamming']"
        super().__init__(self.message)
        
class InvalidNodeStateError(Error):
    """
    Node only have four possible state status:
    ['Red', 'Blue', 'Block', None]
    """
    
    def __init__(self):
        self.message = "Node only have four possible state status: ['Red', 'Blue', 'Block', None]"
        super().__init__(self.message)
        
class InvalidSearchPointError(Error):
    """
    Current AStar Start point or Goal point is a board obstacle which is invalid for path finding.
    """
    
    def __init__(self, board):
        self.message = (
            "Current AStar Start point or Goal point is a board obstacle which is invalid for path finding.\n",
            "Start and Goal points cannot be one of the following coordinates:\n",
            f"{board.barriers}"
        ) 
        super().__init__(self.message)
        
class InvalidStartError(Error):
    """
    Current start point out of board existing board dimension.
    """
    
    def __init__(self):
        self.message ="Current start point out of board existing board dimension."

        super().__init__(self.message)
        
class InvalidGoalError(Error):
    """
    Current goal point out of board existing board dimension.
    """
    
    def __init__(self):
        self.message ="Current goal point out of board existing board dimension."

        super().__init__(self.message)

### Define HexNode type for Hexagon Cell

In [62]:
# define the node class
class HexNode:
    """
    In Cachex game, each Hexagon cell will be represented with a object HexNode,
    where HexNode contains its coordinates, next valid moves, current hexagon cell 
    status.
    input: coords: tuple, move: list, state: string or None
    return: class HexNode
    """
    def __init__(self, coord: tuple, state=None, board=None):
        self.coord = coord
        
        # if board is known, then automatically generate next moves
        if board:
            self.next = self.find_next_moves(board=board, inplace=False)
        else:
            self.next = set()
        
        if state not in ['Red', 'Blue', 'Block', None]:
            raise InvalidNodeStateError
        self.state = None # state could be Red, Blue, Block or None
        
    def summary(self):
        """
        Print Node object detailed information.
        """
        print("=====================================================================")
        print(f"Current Node Position: {self.coord}")
        print(f"Current Node State: {self.state}")
        print(f"Next Possible Moves: {self.next}")
        print("=====================================================================")
        
        
    def distance_diff(self, point1: tuple, point2: tuple, heuristic='manhatten', p=None):
        """
        Calculate the distance between current node and the target hexagon cell using
        given heuristic distance function
        
        heuristic must be one of the following distance formula:
        ['euclidean', 'manhatten', 'hamming']
        """
        if heuristic not in ['euclidean', 'manhatten', 'minkowski']:
            raise InvalidHeuristicError
        
        # calculate the distance with the given heuristic distance formula
        if heuristic is 'euclidean':
            return self.minkowski(point1, point2, 2)
        elif heuristic is 'manhatten':
            return self.minkowski(point1, point2, 1)
        elif heuristic is 'minkowski':
            return self.minkowski(point1, point2, p)
    
    def minkowski(self, point1: tuple, point2: tuple, p:int):
        """
        Calculate the distance use minkowski distance formula where
        distance = (sum( abs(point1[0] - point2[0])^p, abs(point1[1] - point2[1])^p ))**(1/p)
        
        where p = 1, minkowski == manhatten distance
        where p = 2, minkowski == euclidean distance
        """
        return pow(pow(abs(point1[0] - point2[0]), p) + pow(abs(point1[1]-point2[1]), p), 1/p)
    
    def find_next_moves(self, board: CachexBoard, inplace=True, verbose=0):
        """
        Through obversation, if turn the Cachex game board from a hexagon 2D layout to a rectangle grid
        layout, a node could move in six directions except a node cannot move along the major axis.
        
        Hence for each point:
        1. check all posible moves
        2. check moves are vaild (in board)
        3. return a result list
        
        board: set # game board coordinates information to check a move is valid or not
        inplace: boolean # could return a set instead changing node attribute value
        """
        
        # define the output result
        possible_moves = set()
        
        # generate possible moves
        for r in range(self.coord[0]-1, self.coord[0]+2):
            for q in range(self.coord[1]-1,  self.coord[1]+2):
                if (r, q) in board:
                    if verbose == 1:
                        print(f"Expanding node: {self.coord}, generated next valid move {(r, q)}")
                    possible_moves.add((r, q))

        # remove the diagonal elements along the major axis
        if verbose == 1:
            print(f"Removing diagonal element {(self.coord[0]-1, self.coord[1]-1)}, {(self.coord[0]+1, self.coord[1]+1)}")
        if (self.coord[0]-1, self.coord[1]-1) in possible_moves:
            possible_moves.remove((self.coord[0]-1, self.coord[1]-1))
        if (self.coord[0]+1, self.coord[1]+1) in possible_moves:
            possible_moves.remove((self.coord[0]+1, self.coord[1]+1))
        if self.coord in possible_moves:
            possible_moves.remove(self.coord)
        
        
        # return result
        if inplace is True:
            self.next = possible_moves
            return
        
        return possible_moves
    
    def AStar_shortest_path_to(self, target: tuple, board: CachexBoard, heuristic='manhatten', p=None, filename=None):
        """
        A* Search Algorithm implmentation.
        Find the closest path from the current point to the destination point.
        
        Logic:
        - check start and end are not blocks, if blocked then raise error
        - if target has the same coordinates with node itself, then return a dictionary
            Example
                {
                    'step': 8,
                    'path': [(4, 2), (4, 1), (3, 1), (2, 1), (1, 2), (0, 2), (0, 1), (0, 0)]
                }
        - AStar path finding:
            - create a path list to store processed path
            - create a set to store all visited path
            
            1. calculate the distance between each next move and the goal
               where next move nodes have not visited before
            2. append minimum distant node to processed path list
            3. append minimum distant node to visited path set
            3. recursive until reach the goal otherwise
                if node.next is empty:
                    pop the last item in path list and redo from step 1
        """
        # check start and target goal states which are not Blocks
        if self.coord in board.barriers or target in board.barriers:
            raise InvalidSearchPointError(board)
        
        # check start and target goal at the same location 
        if self.coord is target:
            return {'step': 1, 'path': [self.coord]}
        
        path, visited = [self.coord], set()
        
        # if last item in path is the end, return the path
        # else find the path
 
        while path[-1] != target and path != []:
            current_node = board.NodeDict[path[-1]]
       
            # calculate the distance of between all next possible moves and target
            # distance[0][0] is the next best move distance to goal 
            # distance[0][1] is the next best move direction
            distance = self.next_valid_node_distance(current_node.next, target, 
                                                     visited, board, heuristic, p) 
            
            # always move to the direction has the lowest cost
            # if no further path available, then set 1 step back
            visited.add(current_node.coord)
        
            if distance == []:
                del path[-1]

            # if the next move has a smaller distance then update the path
            else:
                minimum_distant_node = distance[0][1]
                path.append(minimum_distant_node)
        
        if filename is not None:
            self.to_txt(path_result={'step': len(path), 'path': path}, filename=filename)
    
        # update the result
        return {'step': len(path), 'path': path}
    
    def next_valid_node_distance(self, nodes, target, visited, board, heuristic='manhatten', p=None):
        """
        Return a list of distance between nodes and their next valid nodes
        """
        distance = []
        for node in nodes:
            if node not in board.barriers and node not in visited:
                dist = self.distance_diff(node, target, heuristic, p)
                distance.append((dist, node))
    
        return sorted(distance)
    
    def state_check(self, isState):
        """
        Return current state check result by compare the target value
        ['Red', 'Blue', 'Block', None]
        return boolean status True or False
            if is None, then the next move is valid
            if not then the next move is invalid
        """
        if isState not in ['Red', 'Blue', 'Block', None]:
            raise InvalidNodeStateError
        
        if self.state == isState:
            return True
        return False
    
    def to_txt(self, path_result, filename):
        """
        Convert path result to txt file
        """
        with open(filename, 'w') as f:
            f.write(str(path_result['step']) + '\n')
            for node in path_result['path']:
                f.write(str(node) + '\n')

### Generate path finding result as a new txt file

In [57]:
CUSTOM_BOARD_DATA = {
    'n': 5,
    'board': [['b', 1, 0], ['b', 1, 1], ['b', 3, 2], ['b', 1, 3]],
    'start': [4, 2],
    'goal': [0, 0]
}


# approach 1: generate file through AStar function
node.AStar_shortest_path_to(target=(CUSTOM_BOARD_DATA['goal'][0], 
                                            CUSTOM_BOARD_DATA['goal'][1]), 
                                    board=board, heuristic='euclidean', filename='sample_output_2.txt')

# approach 2: generate file through node feature .to_txt
node.to_txt(node.AStar_shortest_path_to(target=(CUSTOM_BOARD_DATA['goal'][0], 
                                            CUSTOM_BOARD_DATA['goal'][1]), 
                                    board=board, heuristic='euclidean'), 'sample_output_1.txt', )

### Testing

In [63]:
CUSTOM_BOARD_DATA = {
    'n': 5,
    'board': [['b', 1, 0], ['b', 1, 1], ['b', 3, 2], ['b', 1, 3]],
    'start': [4, 2],
    'goal': [0, 0]
}

# initialise board
board = CachexBoard(CUSTOM_BOARD_DATA)
node = HexNode((CUSTOM_BOARD_DATA['start'][0], 
                CUSTOM_BOARD_DATA['start'][1]), 
               board=board.board)

# create path_dict
path = node.AStar_shortest_path_to(target=(CUSTOM_BOARD_DATA['goal'][0], 
                                            CUSTOM_BOARD_DATA['goal'][1]), 
                                    board=board, heuristic='euclidean')['path']
board.display(path)

2022 COMP30024 Artificial Intelligence Cachex Game
                       Group 4399 S Huang, W, Zhao
--------------------------------------------------
Symbol Representation:
- Δ: AStar Search Start Point
- $: AStar Search End/Goal Point
- #: Barriers, node cannot place at here
- [1-8]: AStar Path Result
--------------------------------------------------
Board Information: Start: (4, 2) >>> End: (0, 0)
- AStar Path Length: 8
--------------------------------------------------
AStar Path:
Start -> (4, 1) -> (3, 1) -> (2, 1) -> (1, 2) ->
(0, 2) -> (0, 1) -> Goal
--------------------------------------------------
             .-'-._.-'-._.-'-._.-'-._.-'-.
            |     |  2  |  Δ  |     |     |
          .-'-._.-'-._.-'-._.-'-._.-'-._.-'
         |     |  3  |  #  |     |     |
       .-'-._.-'-._.-'-._.-'-._.-'-._.-'
      |     |  4  |     |     |     |
    .-'-._.-'-._.-'-._.-'-._.-'-._.-'
   |  #  |  #  |  5  |  #  |     |
 .-'-._.-'-._.-'-._.-'-._.-'-._.-'
|  $  |  7  |  6  |    

In [33]:
CUSTOM_BOARD_DATA = {
    'n': 5,
    'board': [['b', 1, 0], ['b', 1, 1], ['b', 3, 2], ['b', 1, 3]],
    'start': [4, 2],
    'goal': [1, 4]
}

# initialise board
board = CachexBoard(CUSTOM_BOARD_DATA)
node = HexNode((CUSTOM_BOARD_DATA['start'][0], 
                CUSTOM_BOARD_DATA['start'][1]), 
               board=board.board)

# create path_dict
path = node.AStar_shortest_path_to(target=(CUSTOM_BOARD_DATA['goal'][0], 
                                            CUSTOM_BOARD_DATA['goal'][1]), 
                                    board=board, heuristic='euclidean')['path']

board.display(path)

2022 COMP30024 Artificial Intelligence Cachex Game
                       Group 4399 S Huang, W, Zhao
--------------------------------------------------
Symbol Representation:
- Δ: AStar Search Start Point
- $: AStar Search End/Goal Point
- #: Barriers, node cannot place at here
- [1-4]: AStar Path Result
--------------------------------------------------
Board Information: Start: (4, 2) >>> End: (1, 4)
- AStar Path Length: 4
--------------------------------------------------
AStar Path:
Start -> (3, 3) -> (2, 4) -> Goal
--------------------------------------------------
             .-'-._.-'-._.-'-._.-'-._.-'-.
            |     |     |  Δ  |     |     |
          .-'-._.-'-._.-'-._.-'-._.-'-._.-'
         |     |     |  #  |  2  |     |
       .-'-._.-'-._.-'-._.-'-._.-'-._.-'
      |     |     |     |     |  3  |
    .-'-._.-'-._.-'-._.-'-._.-'-._.-'
   |  #  |  #  |     |  #  |  $  |
 .-'-._.-'-._.-'-._.-'-._.-'-._.-'
|     |     |     |     |     |
'-._.-'-._.-'-._.-'-._.-'-._.-'

In [34]:
CUSTOM_BOARD_DATA = {
    'n': 5,
    'board': [['b', 1, 0], ['b', 1, 1], ['b', 3, 2], ['b', 1, 3]],
    'start': [4, 0],
    'goal': [0, 4]
}

# initialise board
board = CachexBoard(CUSTOM_BOARD_DATA)
node = HexNode((CUSTOM_BOARD_DATA['start'][0], 
                CUSTOM_BOARD_DATA['start'][1]), 
               board=board.board)

# create path_dict
path = node.AStar_shortest_path_to(target=(CUSTOM_BOARD_DATA['goal'][0], 
                                            CUSTOM_BOARD_DATA['goal'][1]), 
                                    board=board, heuristic='euclidean')['path']

board.display(path)

2022 COMP30024 Artificial Intelligence Cachex Game
                       Group 4399 S Huang, W, Zhao
--------------------------------------------------
Symbol Representation:
- Δ: AStar Search Start Point
- $: AStar Search End/Goal Point
- #: Barriers, node cannot place at here
- [1-6]: AStar Path Result
--------------------------------------------------
Board Information: Start: (4, 0) >>> End: (0, 4)
- AStar Path Length: 6
--------------------------------------------------
AStar Path:
Start -> (3, 1) -> (2, 2) -> (1, 2) -> (0, 3) ->
Goal
--------------------------------------------------
             .-'-._.-'-._.-'-._.-'-._.-'-.
            |  Δ  |     |     |     |     |
          .-'-._.-'-._.-'-._.-'-._.-'-._.-'
         |     |  2  |  #  |     |     |
       .-'-._.-'-._.-'-._.-'-._.-'-._.-'
      |     |     |  3  |     |     |
    .-'-._.-'-._.-'-._.-'-._.-'-._.-'
   |  #  |  #  |  4  |  #  |     |
 .-'-._.-'-._.-'-._.-'-._.-'-._.-'
|     |     |     |  5  |  $  |
'-._.-'-._.

In [35]:
CUSTOM_BOARD_DATA = {
    'n': 5,
    'board': [['b', 1, 0], ['b', 1, 1], ['b', 3, 2], ['b', 1, 3], ['b', 0, 3]],
    'start': [0, 0],
    'goal': [0, 4]
}

# initialise board
board = CachexBoard(CUSTOM_BOARD_DATA)
node = HexNode((CUSTOM_BOARD_DATA['start'][0], 
                CUSTOM_BOARD_DATA['start'][1]), 
               board=board.board)

# create path_dict
path = node.AStar_shortest_path_to(target=(CUSTOM_BOARD_DATA['goal'][0], 
                                            CUSTOM_BOARD_DATA['goal'][1]), 
                                    board=board, heuristic='euclidean')['path']

board.display(path=path)

2022 COMP30024 Artificial Intelligence Cachex Game
                       Group 4399 S Huang, W, Zhao
--------------------------------------------------
Symbol Representation:
- Δ: AStar Search Start Point
- $: AStar Search End/Goal Point
- #: Barriers, node cannot place at here
- [1-8]: AStar Path Result
--------------------------------------------------
Board Information: Start: (0, 0) >>> End: (0, 4)
- AStar Path Length: 8
--------------------------------------------------
AStar Path:
Start -> (0, 1) -> (0, 2) -> (1, 2) -> (2, 2) ->
(2, 3) -> (1, 4) -> Goal
--------------------------------------------------
             .-'-._.-'-._.-'-._.-'-._.-'-.
            |     |     |     |     |     |
          .-'-._.-'-._.-'-._.-'-._.-'-._.-'
         |     |     |  #  |     |     |
       .-'-._.-'-._.-'-._.-'-._.-'-._.-'
      |     |     |  5  |  6  |     |
    .-'-._.-'-._.-'-._.-'-._.-'-._.-'
   |  #  |  #  |  4  |  #  |  7  |
 .-'-._.-'-._.-'-._.-'-._.-'-._.-'
|  Δ  |  2  |  3  |  # 